In [1]:
import numpy as np
import os
import pickle
import pandas as pd
from scipy.special import softmax

In [2]:
output_dir = "/vision/asomaya1/epic-sounds-annotations/src/outputs"
mode = "val" #options: {val, test} NOTE: test cannot be analyzed as we don't have GT - it is for the EPIC-KITCHENS challenge.
if mode == "test":
    output_file = "scores/EPIC_Sounds_recognition_test_timestamps.pkl"
else:
    output_file = "scores/EPIC_Sounds_validation.pkl"
val_annotations_file = "/vision/asomaya1/epic-sounds-annotations/EPIC_Sounds_validation.pkl"

In [3]:
with open(os.path.join(output_dir, mode, output_file), 'rb') as f:
    pred_annotations = pickle.load(f)

In [4]:
val_annotations = pd.read_pickle(val_annotations_file)

In [19]:
probabilities = softmax(pred_annotations['interaction_output'], axis=1)

# Get the indices of the top 5 predictions for each sample
top5_predicted_classes = np.argsort(-probabilities, axis=1)[:, :5]

# Predicted classes (top-1)
predicted_classes = top5_predicted_classes[:, 0]

# Create a DataFrame with predictions
df_predictions = pd.DataFrame({
    'annotation_id': pred_annotations['annotation_id'],
    'pred_class': predicted_classes,
    'top5_classes': list(top5_predicted_classes)
})

# Merge with ground truth annotations
merged_df = pd.merge(df_predictions, val_annotations[['annotation_id', 'class_id']], on='annotation_id')

# Compute top-1 accuracy
top1_accuracy = np.mean(merged_df['pred_class'] == merged_df['class_id'])

# Compute top-5 accuracy
top5_accuracy = merged_df.apply(lambda row: row['class_id'] in row['top5_classes'], axis=1).mean()

In [20]:
top5_accuracy

0.7591785936527692